In [47]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


from sklearn.preprocessing import StandardScaler

from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [48]:
df = pd.read_csv('../datasets/sales_data.csv')

In [49]:
df.head()

,OrderNumber,OrderDate,WarehouseCode,Sales Channel,StoreID,SalesTeamID,CustomerID,ProductID,Order_Quantity,Discount_Applied,Unit_Price,Unit_Cost
0,SO - 000101,5/31/2018,WARE-UHY1004,1,259,6,15,12,5,0.075,1963.1,1001.18
1,SO - 000102,5/31/2018,WARE-NMK1003,2,196,14,20,27,3,0.075,3939.6,3348.66
2,SO - 000103,5/31/2018,WARE-UHY1004,3,213,21,16,16,1,0.050,1775.5,781.22
3,SO - 000104,5/31/2018,WARE-NMK1003,4,107,28,48,23,8,0.075,2324.9,1464.69
4,SO - 000105,5/31/2018,WARE-NMK1003,3,111,22,49,26,8,0.100,1822.4,1476.14


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7991 entries, 0 to 7990
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   OrderNumber       7991 non-null   object 
 1   OrderDate         7991 non-null   object 
 2   WarehouseCode     7991 non-null   object 
 3   Sales Channel     7991 non-null   int64  
 4   StoreID           7991 non-null   int64  
 5   SalesTeamID       7991 non-null   int64  
 6   CustomerID        7991 non-null   int64  
 7   ProductID         7991 non-null   int64  
 8   Order_Quantity    7991 non-null   int64  
 9   Discount_Applied  7991 non-null   float64
 10  Unit_Price        7991 non-null   float64
 11  Unit_Cost         7991 non-null   float64
dtypes: float64(3), int64(6), object(3)
memory usage: 749.3+ KB


In [51]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Sales Channel,7991.0,1.982856,1.015687,1.00,1.00,2.000,3.00,4.00
StoreID,7991.0,183.850081,105.903946,1.00,91.00,183.000,276.00,367.00
SalesTeamID,7991.0,14.384307,7.986086,1.00,8.00,14.000,21.00,28.00
CustomerID,7991.0,25.457014,14.414883,1.00,13.00,25.000,38.00,50.00
ProductID,7991.0,23.771743,13.526545,1.00,12.00,24.000,36.00,47.00
Order_Quantity,7991.0,4.525341,2.312631,1.00,3.00,5.000,7.00,8.00
Discount_Applied,7991.0,0.114394,0.085570,0.05,0.05,0.075,0.15,0.40
Unit_Price,7991.0,2284.536504,1673.096364,167.50,1031.80,1849.200,3611.30,6566.00
Unit_Cost,7991.0,1431.911513,1112.413063,68.68,606.12,1080.580,2040.25,5498.56


In [52]:
df.isnull().values.any()

False

In [53]:
df.columns

Index(['OrderNumber', 'OrderDate', 'WarehouseCode', 'Sales Channel', 'StoreID',
       'SalesTeamID', 'CustomerID', 'ProductID', 'Order_Quantity',
       'Discount_Applied', 'Unit_Price', 'Unit_Cost'],
      dtype='object')

In [54]:
train_cols = ['Sales Channel', 'StoreID',
       'SalesTeamID', 'CustomerID', 'ProductID',
       'Discount_Applied', 'Unit_Price', 'Unit_Cost']

train_cols_few = ['StoreID',
       'SalesTeamID', 'CustomerID', 'ProductID', 'Unit_Price', 'Unit_Cost']

x = df[train_cols_few]
y = df['Order_Quantity']

In [55]:
x

,StoreID,SalesTeamID,CustomerID,ProductID,Unit_Price,Unit_Cost
0,259,6,15,12,1963.1,1001.18
1,196,14,20,27,3939.6,3348.66
2,213,21,16,16,1775.5,781.22
3,107,28,48,23,2324.9,1464.69
4,111,22,49,26,1822.4,1476.14
...,...,...,...,...,...,...
7986,339,9,41,29,234.5,121.94
7987,202,14,29,3,3202.6,1921.56
7988,241,14,32,35,3825.7,2792.76
7989,112,20,42,36,1072.0,804.00


In [56]:
y

0       5
1       3
2       1
3       8
4       8
       ..
7986    1
7987    6
7988    5
7989    8
7990    5
Name: Order_Quantity, Length: 7991, dtype: int64

In [57]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=30)

In [58]:
x_train_scale = StandardScaler().fit_transform(x_train)
x_test_scale = StandardScaler().fit_transform(x_test)

In [59]:
# define the model
model = ExtraTreesRegressor()

In [60]:
# evaluate the model
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

In [61]:
n_scores = cross_val_score(model, x, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')

In [62]:
# report performance
print('MAE: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

MAE: -2.065 (0.035)
